<a href="https://colab.research.google.com/github/vanessabrolim/comfort_analysis/blob/master/HMM_xyz.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pomegranate
import numpy as np
import pandas as pd
from pomegranate import *
from matplotlib import *
import datetime
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive',  force_remount=True)

Mounted at /content/drive


In [2]:
data = pd.read_pickle('drive/My Drive/Udesc/train.pkl')
data.reset_index(inplace=True)
data = data.drop('date_time', 1)
data = data.drop('index', 1)
data = data.drop('comfort_geral', 1)
#data = data.drop('engine_rpm', 1)
#data = data.drop('engine_load', 1)
#data = data.drop('intake_manifold_pressure', 1)
#data = data.drop('accelerator_pedal', 1)
data_acc = pd.read_pickle('drive/My Drive/Udesc/train.pkl')
data_acc.reset_index(inplace=True)
shape = data.shape
data.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x,comfort_y,comfort_z
0,NaN,NaN,NaN,NaN,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0,2.0,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0,4.0,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0,4.0,4.0
3,NaN,NaN,NaN,NaN,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0,3.0,3.0
4,NaN,NaN,NaN,NaN,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0,4.0,4.0


In [3]:
data.isna().sum()

engine_rpm                  6465
engine_load                 6461
intake_manifold_pressure    6465
accelerator_pedal           6460
speed                          0
giroscope_x                    0
giroscope_y                    0
giroscope_z                    0
accelerometer_x_avg            0
accelerometer_y_avg            0
accelerometer_z_avg            0
accelerometer_x_std            0
accelerometer_y_std            0
accelerometer_z_std            0
comfort_x                      0
comfort_y                      0
comfort_z                      0
dtype: int64

In [4]:
data.describe()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x,comfort_y,comfort_z
count,1777.000000,1781.000000,1777.000000,1782.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000,8242.000000
mean,1233.486699,26.227108,136.271720,32.786051,14.483692,-1.168270,-0.240637,-2.410170,1.015711,-0.022959,0.127480,0.079423,0.063378,0.070610,3.324800,3.743752,3.743752
std,341.504232,25.307262,38.331160,31.740511,16.268199,2.237187,2.420126,1.846004,0.005636,0.065062,0.018262,0.071321,0.058779,0.060056,0.767429,0.594927,0.594927
min,693.694511,0.000000,97.891304,0.000000,0.001048,-25.270000,-28.960000,-17.210000,0.984244,-0.125264,0.070017,0.002721,0.003038,0.004281,2.000000,2.000000,2.000000
25%,834.918660,9.078269,105.550239,0.000000,0.074244,-1.500000,-0.640000,-2.660000,1.013707,-0.070896,0.114836,0.003992,0.003739,0.004998,3.000000,4.000000,4.000000
50%,1265.206813,15.172646,124.087678,31.753677,6.605771,-1.220000,-0.240000,-2.380000,1.017197,-0.056405,0.122441,0.070782,0.056756,0.069267,3.000000,4.000000,4.000000
75%,1529.489051,40.608780,151.673031,53.064070,30.373221,-0.880000,0.160000,-2.140000,1.017829,0.070473,0.138814,0.127829,0.104235,0.115893,4.000000,4.000000,4.000000
max,2130.300231,99.600000,255.000000,100.000000,56.231634,23.740000,40.870000,22.370000,1.039291,0.105421,0.186455,0.312701,0.243907,0.264738,5.000000,5.000000,5.000000


In [0]:
columns = ['engine_rpm', 'engine_load', 'intake_manifold_pressure', 'accelerator_pedal', 'speed', 'giroscope_x', 'giroscope_y', 'giroscope_z']
#columns = ['engine_rpm', 'speed', 'giroscope_x', 'giroscope_y', 'giroscope_z']

# DataSets eixo X

In [0]:
dx = data.copy()
dx = dx.drop('comfort_y', 1)
dx = dx.drop('comfort_z', 1)

In [7]:
dx_mean = dx.copy()
for c in columns:
  dx_mean[c] = dx_mean[c].fillna(dx[c].mean())
dx_mean.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x
0,1233.486699,26.227108,136.271720,32.786051,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0
3,1233.486699,26.227108,136.271720,32.786051,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1233.486699,26.227108,136.271720,32.786051,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [8]:
dx_median = dx.copy()

for c in columns:
  dx_median[c] = dx_median[c].fillna(dx[c].median())
dx_median.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x
0,1265.206813,15.172646,124.087678,31.753677,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0
3,1265.206813,15.172646,124.087678,31.753677,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1265.206813,15.172646,124.087678,31.753677,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [9]:
dx_max = dx.copy()

for c in columns:
  dx_max[c] = dx_max[c].fillna(dx[c].max())
dx_max.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x
0,2130.300231,99.600000,255.000000,100.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0
3,2130.300231,99.600000,255.000000,100.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,2130.300231,99.600000,255.000000,100.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [10]:
dx_min = dx.copy()

for c in columns:
  dx_min[c] = dx_min[c].fillna(dx[c].min())
dx_min.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x
0,693.694511,0.000000,97.891304,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0
3,693.694511,0.000000,97.891304,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,693.694511,0.000000,97.891304,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [11]:
dx_zeros = dx.copy()

for c in columns:
  dx_zeros[c] = dx_zeros[c].fillna(0)
dx_zeros.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_x
0,0.000000,0.000000,0.000000,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,3.0
3,0.000000,0.000000,0.000000,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,0.000000,0.000000,0.000000,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


# DataSets eixo Y

In [0]:
dy = data.copy()
dy = dy.drop('comfort_x', 1)
dy = dy.drop('comfort_z', 1)

In [13]:
dy_mean = dy.copy()

for c in columns:
  dy_mean[c] = dy_mean[c].fillna(dy[c].mean())
dy_mean.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_y
0,1233.486699,26.227108,136.271720,32.786051,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,1233.486699,26.227108,136.271720,32.786051,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1233.486699,26.227108,136.271720,32.786051,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [14]:
dy_median = dy.copy()

for c in columns:
  dy_median[c] = dy_median[c].fillna(dy[c].median())
dy_median.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_y
0,1265.206813,15.172646,124.087678,31.753677,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,1265.206813,15.172646,124.087678,31.753677,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1265.206813,15.172646,124.087678,31.753677,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [15]:
dy_zeros = dy.copy()

for c in columns:
  dy_zeros[c] = dy_zeros[c].fillna(0)
dy_zeros.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_y
0,0.000000,0.000000,0.000000,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,0.000000,0.000000,0.000000,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,0.000000,0.000000,0.000000,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [16]:
dy_max = dy.copy()

for c in columns:
  dy_max[c] = dy_max[c].fillna(dy[c].max())
dy_max.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_y
0,2130.300231,99.600000,255.000000,100.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,2130.300231,99.600000,255.000000,100.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,2130.300231,99.600000,255.000000,100.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [17]:
dy_min = dy.copy()

for c in columns:
  dy_min[c] = dy_min[c].fillna(dy[c].min())
dy_min.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_y
0,693.694511,0.000000,97.891304,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,693.694511,0.000000,97.891304,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,693.694511,0.000000,97.891304,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


# DataSets eixo Z

In [0]:
dz = data.copy()
dz = dz.drop('comfort_x', 1)
dz = dz.drop('comfort_y', 1)

In [19]:
dz_mean = dz.copy()

for c in columns:
  dz_mean[c] = dz_mean[c].fillna(dz[c].mean())
dz_mean.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_z
0,1233.486699,26.227108,136.271720,32.786051,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,1233.486699,26.227108,136.271720,32.786051,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1233.486699,26.227108,136.271720,32.786051,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [20]:
dz_zeros = dz.copy()

for c in columns:
  dz_zeros[c] = dz_zeros[c].fillna(0)
dz_zeros.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_z
0,0.000000,0.000000,0.000000,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,0.000000,0.000000,0.000000,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,0.000000,0.000000,0.000000,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [21]:
dz_median = dz.copy()

for c in columns:
  dz_median[c] = dz_median[c].fillna(dz[c].median())
dz_median.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_z
0,1265.206813,15.172646,124.087678,31.753677,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,1265.206813,15.172646,124.087678,31.753677,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,1265.206813,15.172646,124.087678,31.753677,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [22]:
dz_max = dz.copy()

for c in columns:
  dz_max[c] = dz_max[c].fillna(dz[c].max())
dz_max.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_z
0,2130.300231,99.600000,255.000000,100.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,2130.300231,99.600000,255.000000,100.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,2130.300231,99.600000,255.000000,100.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


In [23]:
dz_min = dz.copy()

for c in columns:
  dz_min[c] = dz_min[c].fillna(dz[c].min())
dz_min.head()

,engine_rpm,engine_load,intake_manifold_pressure,accelerator_pedal,speed,giroscope_x,giroscope_y,giroscope_z,accelerometer_x_avg,accelerometer_y_avg,accelerometer_z_avg,accelerometer_x_std,accelerometer_y_std,accelerometer_z_std,comfort_z
0,693.694511,0.000000,97.891304,0.000000,23.223407,-2.72,3.59,-0.34,1.016963,-0.059846,0.120445,0.194853,0.155698,0.172987,2.0
1,1034.423358,55.391512,111.343066,92.270318,3.223987,-0.77,-0.31,-4.57,1.005749,-0.099241,0.110642,0.217798,0.205179,0.219697,4.0
2,1487.816867,9.590048,118.028169,21.461805,22.807823,-2.05,0.98,4.51,1.024716,-0.063518,0.128107,0.109880,0.089158,0.083585,4.0
3,693.694511,0.000000,97.891304,0.000000,8.774839,-2.20,0.19,-2.51,1.015846,-0.079579,0.139395,0.035262,0.018842,0.042641,3.0
4,693.694511,0.000000,97.891304,0.000000,13.601787,-1.16,-0.10,-2.03,1.012846,-0.077933,0.137599,0.048181,0.030723,0.042666,4.0


# Treinos X

## DataSet puro

In [0]:
shape = dx.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_zeros)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
model = HiddenMarkovModel()

In [0]:
model.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  model.add_transition(model.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    model.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  model.add_transition(states[i], model.end, 0.2)

In [0]:
model.bake()

In [30]:
model.dense_transition_matrix()

array([[0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.        , 0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.        , 0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.        , 0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.        , 0.16666667],
       [0.16666667, 0.16666667, 0.16666667, 0.16666667, 0.16666667,
        0.        , 0.16666667],
       [0.2       , 0.2       , 0.2       , 0.2       , 0.2       ,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        ]])

In [31]:
model.predict(dx.iloc[1])


[0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

In [0]:
b = numpy.array(dx)

In [0]:
model.fit(b, verbose = True)

In [0]:
model.dense_transition_matrix()

In [0]:
model.predict(dx.iloc[1])

In [0]:
model_dx = model.to_json()


## DataSet Mean

In [0]:
shape = dx_mean.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_mean)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
mx_mean = HiddenMarkovModel()

In [0]:
mx_mean.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  mx_mean.add_transition(mx_mean.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    mx_mean.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  mx_mean.add_transition(states[i], mx_mean.end, 0.2)

In [0]:
mx_mean.bake()

In [0]:
mx_mean.dense_transition_matrix()

In [0]:
mx_mean.predict(dx.iloc[1])


In [0]:
b = numpy.array(dx_mean)

In [0]:
mx_mean.fit(b, verbose = False)

In [0]:
mx_mean.dense_transition_matrix()

In [0]:
mx_mean.predict(dx_mean.iloc[1])

In [0]:
mx_mean = mx_mean.to_json()


## DataSet Median

In [0]:
shape = dx_median.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_median)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
mx_median = HiddenMarkovModel()

In [0]:
mx_median.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  mx_mean.add_transition(model.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    mx_mean.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  mx_mean.add_transition(states[i], model.end, 0.2)

In [0]:
mx_median.bake()

In [0]:
mx_median.dense_transition_matrix()

In [0]:
mx_median.predict(dx_median.iloc[1])


In [0]:
b = numpy.array(dx_median)

In [0]:
mx_median.fit(b, verbose = False)

In [0]:
mx_median.dense_transition_matrix()

In [0]:
mx_median.predict(dx_median.iloc[1])

In [0]:
mx_median = mx_median.to_json()


## DataSet Zeros

In [0]:
shape = dx_zeros.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_zeros)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
mx_zeros = HiddenMarkovModel()

In [0]:
mx_zeros.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  mx_zeros.add_transition(mx_zeros.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    mx_zeros.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  mx_zeros.add_transition(states[i], mx_zeros.end, 0.2)

In [0]:
mx_zeros.bake()

In [0]:
mx_zeros.dense_transition_matrix()

In [0]:
mx_zeros.predict(dx_zeros.iloc[1])


In [0]:
b = numpy.array(dx_zeros)

In [0]:
mx_zeros.fit(b, verbose = False)

In [0]:
mx_zeros.dense_transition_matrix()

In [0]:
mx_zeros.predict(dx_zeros.iloc[1])

In [0]:
mx_zeros = mx_zeros.to_json()


## DataSet Max

In [0]:
shape = dx_max.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_max)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
mx_max = HiddenMarkovModel()

In [0]:
mx_max.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  mx_max.add_transition(mx_max.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    mx_max.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  mx_max.add_transition(states[i], model.mx_max, 0.2)

In [0]:
mx_max.bake()

In [0]:
mx_max.dense_transition_matrix()

In [0]:
mx_max.predict(dx_median.iloc[1])


In [0]:
b = numpy.array(dx_max)

In [0]:
dx_max.fit(b, verbose = False)

In [0]:
mx_max.dense_transition_matrix()

In [0]:
mx_max.predict(dx_max.iloc[1])

In [0]:
mx_max = mx_max.to_json()



## DataSet Min

In [0]:
shape = dx_min.shape
cov = numpy.eye(shape[1])
kmodel = KMeans(5).fit(dx_min)
mu = kmodel.cluster_centers_

In [0]:
s1 = State(MultivariateGaussianDistribution(mu[0], cov))
s2 = State(MultivariateGaussianDistribution(mu[1], cov))
s3 = State(MultivariateGaussianDistribution(mu[2], cov))
s4 = State(MultivariateGaussianDistribution(mu[3], cov))
s5 = State(MultivariateGaussianDistribution(mu[4], cov))

In [0]:
mx_min = HiddenMarkovModel()

In [0]:
mx_min.add_states(s1, s2, s3, s4, s5)

In [0]:
states = [s1, s2, s3, s4, s5]


for i in range(0,5):
  mx_mean.add_transition(model.start, states[i], 0.2)

for idx, val in enumerate(states):
  for i in range(0,5):
    mx_mean.add_transition(states[idx], states[i], 0.2)
    
for i in range(0,5):
  mx_mean.add_transition(states[i], model.end, 0.2)

In [0]:
mx_min.bake()

In [0]:
mx_min.dense_transition_matrix()

In [0]:
mx_min.predict(dx_median.iloc[1])


In [0]:
b = numpy.array(dx_min)

In [0]:
mx_min.fit(b, verbose = False)

In [0]:
mx_min.dense_transition_matrix()

In [0]:
mx_min.predict(dx_min.iloc[1])

In [0]:
mx_min = mx_min.to_json()


## Gráficos

In [0]:
plt.plot(dx.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")


In [0]:
plt.plot(dx_mean.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")

In [0]:
plt.plot(dx_median.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")

In [0]:
plt.plot(dx_zeros.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")

In [0]:
plt.plot(dx_max.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")

In [0]:
plt.plot(dx_min.T,c='k',lw=0.005) #the data
for i in range(0, 5): #the model means
    plt.scatter(model.states[i].distribution.mu[0],
                model.states[i].distribution.mu[4],
                c='r',
                label="HMM clusters upon convergence")
    plt.scatter(kmodel.cluster_centers_[i][0],
                kmodel.cluster_centers_[i][shape[1]-1],
                c='g',
                label="k_means initial clusters")

# Treinos Y

# Treinos Z